<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/City_Head_Review_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**⏯ Authorization & Import Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
import dateutil.parser
import re
from datetime import date

# from dateutil.relativedelta import relativedelta
from functools import reduce
from datetime import datetime
from datetime import date, timedelta

from google.colab import drive
drive.mount('/content/drive', force_remount = True)


import gspread
from google.colab import auth
# auth.authenticate_user()

# from oauth2client.client import GoogleCredentials
# gc = gspread.authorize(GoogleCredentials.get_application_default())
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#get data from spreadsheetID and Range Name function
def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

Mounted at /content/drive


In [ ]:
def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def parseDateTimeFormatTZ(row):
  return dateutil.parser.parse(row)

def cleanText(row):
  return row.str.upper().str.strip()

def percentage(row):
  return (row *100).round(decimals=2).astype(str) + '%'


def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")

  for (root, dirs, files) in os.walk(file_path):
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()

        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
          temp = pd.read_csv(path,parse_dates=True)
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge


#Review Sheet

In [ ]:
month = 3  # Change month here
year = 2024 # Change year here

target_collection = {10:'1q49Be555U23sUyJoXHmoDCZKpX2VyetlKzmeyPZqTJU',11:'1DsVX4ZwwSSn8RPxaOtNYHM9QZhzoJF3zQ5P5Vq5Td7o',12:'1Vl3jlxNDZW3VGluye-fkDjhcSSnVKNOGcYhlUURz_48',1:'1YvcNKdf4Z389PL2wSB7cyxyGSdoYmUESy9UqVv-Y2cc',2:'1AyWtzY5eyR2a-rzT0Eh7GCfqrsF08tmby4MoLrhz0gU',3:'1SyQlFj_xIUprnw0urvjcvsYhrABStAgJRCmvee2pyK8'}
non_compliant_list = {10:'1Z-9xFagPVvKAIVqFlihHCpQPmmKbgYn43xwexN63Ock',11:'11pFIYCnJEx_w4VRKaIHgIwV6ezrsz276ShF4g_kpPo4',12:'1K2D78I9L-RQYCOB4UFZNnCv6U7UFdotWfTa7WnrM34w',1:'1XDSV3vR7Ct-Ph9PFxwikYdG54DtoU9ORMHOXhzVd1UA',2:'1Uh6Z12bZPeaE5vvZg5anKHYVjvN2PlYfrGb3-LsYPvs',3:'1lsixPueW19zi6JeJE6Hrgo9QcfqQhPaU8ybHot_w7m4'}


#**Overall Productivity**

In [ ]:
# Importing Datasets
# City Target collection Sheet
key = target_collection[month] #Sheet ID need to be changed on every month
range = "PAN India Target"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
target_sheet = get_data_df(key, range)
tsm_aasm = target_sheet[target_sheet['Role'].isin(['TSM','Senior TSM','AASM','AASM IC'])]
asm = target_sheet[target_sheet['Role'].isin(['ASM','Senior ASM'])]
dch = target_sheet[target_sheet['Role'].isin(['DCH'])]
count_tsm = tsm_aasm[['Location','Emp Code']].groupby('Location').count().reset_index()
count_asm = asm[['Location','Emp Code']].groupby('Location').count().reset_index()
count_dch = dch[['Location','Emp Code']].groupby('Location').count().reset_index()
count = pd.merge(pd.merge(count_tsm,count_asm,on='Location',how='outer'),count_dch,on='Location',how='outer').fillna(0)
count.rename(columns={'Emp Code_x':'TSMs + AASMs at start of the month (on Field)','Emp Code_y':'ASMs at start of the month','Emp Code':'DCHs at start of the month'},inplace=True)
count

,Location,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month
0,Ahmedabad,6,0.0,1.0
1,Bangalore,15,3.0,1.0
2,Chennai,4,0.0,0.0
3,Gurgaon,13,0.0,0.0
4,Hyderabad,11,2.0,1.0
5,Kolkata,1,0.0,0.0
6,Mumbai,19,3.0,0.0
7,Navi Mumbai,9,1.0,0.0
8,Noida,13,1.0,0.0
9,Pune,20,3.0,0.0


In [ ]:
key = "1EcV-4vhUf9hpMEWP1hpo6Jl3FU8O3VEFani7l47v8VM"
range = "CH-Achieved"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
ch_sheet = get_data_df(key, range)
ch_sheet = ch_sheet.iloc[:,31:40]
ch_sheet.columns = ch_sheet.iloc[0,:]
ch_sheet = ch_sheet.drop(0)
ch_sheet['CH'] = ch_sheet['CH'].replace("-",np.nan)
ch_sheet['CH'] = ch_sheet['CH'].replace("",np.nan)

ch_sheet

,,Year,Month,CH,City,Target,Total HO IP,Total D1 IP,Flat Count
1,,2021,10,ajaykumar.g@nobroker.in,Bangalore,19000,11468.3,6496.05,7269
2,,2021,10,NaN,Chennai,7349,967.7,1034.45,631
3,,2021,10,manish.chanchal@nobroker.in,Gurgaon,23500,12778,12465.75,7790
4,,2021,10,hemanth.sai@nobroker.in,Hyderabad,12918,3326.675,3262.65625,1520
5,,2021,10,vivek.nikam@nobroker.in,Mumbai,17500,5126.6,4894.8,4246
...,...,...,...,...,...,...,...,...,...
410,,2024,4,suvodeep.sen@nobroker.in,Ahmedabad,700000,34448.0265,0,481
411,,2024,4,NaN,Chennai,450000,106014.125,11613.67,1091
412,,2024,4,ajaykumar.g@nobroker.in,Bangalore South,1200000,-17801.739,0,2715
413,,2024,4,bhushan.s@nobroker.in,Kolkata,300000,6131.805,0,429


In [ ]:
ch_sheet = ch_sheet[(ch_sheet['Month']==str(month)) & (ch_sheet['Year']==str(year))].rename(columns = {'City':'Location'})
ch= ch_sheet[['Location','CH']].groupby('Location').count().reset_index()
count1 = pd.merge(count,ch,how='outer',on='Location').fillna(0)
count1.rename(columns={'CH':'CH Count'},inplace=True)
count1['CHs/DCHs at start of the month']= count1['DCHs at start of the month']+count1['CH Count']
target = pd.merge(ch_sheet[['Location','Target','Total HO IP']],count1,how='outer',on='Location')
target

,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0


In [ ]:
key = "1BBIaBIbkKuQfVYt8W8VwbRPiztg29HPuBHzW764rEwE"
range = "LeadMaster"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
salesmaster = get_data_df(key, range)
salesmaster = salesmaster[salesmaster['Team Category']=='Sales Team']

In [ ]:
#salesmaster['Team Category'].value_counts()

In [ ]:
salesmaster = salesmaster[(salesmaster['Closure Month']==str(month)) & (salesmaster['year']==str(year))]
salesmaster['Sold Flats'] = pd.to_numeric(salesmaster['Sold Flats'])
salesmaster.rename(columns = {'Primary BD City':'Location'},inplace = True)

salesmaster = salesmaster[salesmaster['HO Product']!=""]
total_flats = salesmaster[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(target,total_flats,on='Location',how='inner')
review.rename(columns = {'Sold Flats':'Total flats'},inplace=True)

vms_flats = salesmaster[salesmaster['HO Product'].isin(['VMS only','Both VMS + ERP'])] #Need to select HO product
vms_flats = vms_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,vms_flats,on='Location',how='outer')
review.rename(columns = {'Sold Flats':'VMS Sold flats'},inplace=True)

erp_flats = salesmaster[salesmaster['HO Product'].isin(['ERP only','Both VMS + ERP'])]#Need to select HO product
erp_flats = erp_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,erp_flats,on='Location',how='outer')
review.rename(columns = {'Sold Flats':'ERP Sold flats'},inplace=True)

# new_win_mygate = salesmaster[(salesmaster['VMS Competitor being replaced']=='MyGate') & (salesmaster['Closure Type'].isin(['New Closure','Winback','FC Addon']))]

# new_win_mygate_flats = new_win_mygate[['Location','Sold Flats']].groupby('Location').sum().reset_index()
# review = pd.merge(review,new_win_mygate_flats,on='Location',how='outer')
# review.rename(columns = {'Sold Flats':'New+Winback from MG'},inplace=True)
review


,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0,1772,1725,439.0
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0,2613,1710,1219.0
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0,4434,3765,2579.0
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0,6364,1402,5010.0
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0,10614,9289,1876.0
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0,4075,4075,2472.0
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0,3378,3378,NaN
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0,1343,645,878.0
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0,4702,3483,3541.0
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0,2054,1572,482.0


In [ ]:
new_win_flats = salesmaster[salesmaster['Closure Type'].isin(['New Closure','Winback','FC Addon','Addons'])]
new_win = new_win_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,new_win,on='Location',how='outer')
review.rename(columns = {'Sold Flats':'New+Winback Flats'},inplace=True)

new_win_vms = new_win_flats[new_win_flats['HO Product'].isin(['VMS only'])]
new_win_vms = new_win_vms[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,new_win_vms,on='Location',how='outer')
review.rename(columns = {'Sold Flats':'New+Winback VMS Only'},inplace=True)

new_win_erp = new_win_flats[new_win_flats['HO Product'].isin(['ERP only'])]
new_win_erp = new_win_erp[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,new_win_erp,on='Location',how='outer')
review.rename(columns = {'Sold Flats':'New+Winback ERP Only'},inplace=True)

new_win_vmserp = new_win_flats[new_win_flats['HO Product'].isin(['Both VMS + ERP'])]
new_win_vmserp = new_win_vmserp[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,new_win_vmserp,on='Location',how='outer').fillna(0)
review.rename(columns = {'Sold Flats':'New+Winback Both VMS + ERP'},inplace=True)

review

,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0,1772,1725,439.0,1772,1333,47.0,392.0
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0,2613,1710,1219.0,1768,1394,58.0,316.0
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0,4434,3765,2579.0,4226,1855,461.0,1910.0
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0,6364,1402,5010.0,3532,1301,2183.0,48.0
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0,10614,9289,1876.0,9559,8698,310.0,551.0
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0,4075,4075,2472.0,4075,1603,0.0,2472.0
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0,3378,3378,0.0,3378,3378,0.0,0.0
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0,1343,645,878.0,765,465,120.0,180.0
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0,4702,3483,3541.0,3703,1161,220.0,2322.0
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0,2054,1572,482.0,1572,1572,0.0,0.0


In [ ]:
new_win_builders = new_win_flats[(new_win_flats['Property Type'].str.contains('Builder')==True) | (new_win_flats['Property Type'] == 'Pre Possession (Occupancy less than 2%)')]

new_win_builders = new_win_builders[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,new_win_builders,on='Location',how='outer').fillna(0)
review.rename(columns = {'Sold Flats':'New+Winback Builders'},inplace=True)
review

,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP,New+Winback Builders
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0,1772,1725,439.0,1772,1333,47.0,392.0,0.0
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0,2613,1710,1219.0,1768,1394,58.0,316.0,701.0
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0,4434,3765,2579.0,4226,1855,461.0,1910.0,678.0
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0,6364,1402,5010.0,3532,1301,2183.0,48.0,2119.0
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0,10614,9289,1876.0,9559,8698,310.0,551.0,393.0
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0,4075,4075,2472.0,4075,1603,0.0,2472.0,1474.0
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0,3378,3378,0.0,3378,3378,0.0,0.0,0.0
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0,1343,645,878.0,765,465,120.0,180.0,0.0
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0,4702,3483,3541.0,3703,1161,220.0,2322.0,925.0
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0,2054,1572,482.0,1572,1572,0.0,0.0,0.0


In [ ]:
# new_win_builders['Property Type'].value_counts()

In [ ]:
winback_flats = salesmaster[salesmaster['Closure Type']=='Winback']
winback_flats = winback_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,winback_flats,on='Location',how='outer').fillna(0)
review.rename(columns = {'Sold Flats':'Winback Flats'},inplace=True)

upsell_flats = salesmaster[salesmaster['Closure Type']=='Upsell (VMS/ERP)']
upsell_flats = upsell_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review = pd.merge(review,upsell_flats,on='Location',how='outer').fillna(0)
review.rename(columns = {'Sold Flats':'Upsell Flats'},inplace=True)
review

,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP,New+Winback Builders,Winback Flats,Upsell Flats
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0,1772,1725,439.0,1772,1333,47.0,392.0,0.0,350.0,0.0
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0,2613,1710,1219.0,1768,1394,58.0,316.0,701.0,77.0,845.0
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0,4434,3765,2579.0,4226,1855,461.0,1910.0,678.0,0.0,208.0
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0,6364,1402,5010.0,3532,1301,2183.0,48.0,2119.0,585.0,2832.0
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0,10614,9289,1876.0,9559,8698,310.0,551.0,393.0,2487.0,1055.0
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0,4075,4075,2472.0,4075,1603,0.0,2472.0,1474.0,755.0,0.0
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0,3378,3378,0.0,3378,3378,0.0,0.0,0.0,200.0,0.0
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0,1343,645,878.0,765,465,120.0,180.0,0.0,216.0,578.0
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0,4702,3483,3541.0,3703,1161,220.0,2322.0,925.0,152.0,999.0
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0,2054,1572,482.0,1572,1572,0.0,0.0,0.0,0.0,482.0


In [ ]:
#review1.columns[review1.columns.duplicated()]

In [ ]:
key = "16XZ1WsM2My8xr4zNMkYn_uuqYT_9yUUuTbyNBxMDB4s"
range = "City - Target-Collated"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
city_target = get_data_df(key, range)
city_target = city_target.replace('Bengaluru','Bangalore')

city_target = city_target[(city_target['Month']==str(month)) & (city_target['Year']==str(year))].rename(columns = {'City':'Location'})
review = pd.merge(review,city_target[['Location','Budgeted HC (TSM+AASM)']],on='Location',how='left')
review

,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP,New+Winback Builders,Winback Flats,Upsell Flats,Budgeted HC (TSM+AASM)
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0,1772,1725,439.0,1772,1333,47.0,392.0,0.0,350.0,0.0,NaN
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0,2613,1710,1219.0,1768,1394,58.0,316.0,701.0,77.0,845.0,NaN
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0,4434,3765,2579.0,4226,1855,461.0,1910.0,678.0,0.0,208.0,NaN
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0,6364,1402,5010.0,3532,1301,2183.0,48.0,2119.0,585.0,2832.0,NaN
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0,10614,9289,1876.0,9559,8698,310.0,551.0,393.0,2487.0,1055.0,NaN
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0,4075,4075,2472.0,4075,1603,0.0,2472.0,1474.0,755.0,0.0,NaN
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0,3378,3378,0.0,3378,3378,0.0,0.0,0.0,200.0,0.0,NaN
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0,1343,645,878.0,765,465,120.0,180.0,0.0,216.0,578.0,NaN
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0,4702,3483,3541.0,3703,1161,220.0,2322.0,925.0,152.0,999.0,NaN
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0,2054,1572,482.0,1572,1572,0.0,0.0,0.0,0.0,482.0,NaN


In [ ]:
key = "1iKgiRR-lOQ-zM6v6bh_1P3vkIujVCeBzVVr_nHuoDMU"
range = "Society Master"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
society_master = get_data_df(key, range)
society_master['City'].value_counts()

City
PUNE                   3215
MUMBAI                 2151
BANGALORE              2083
NAVI MUMBAI - THANE    1537
AHMEDABAD              1265
GURGAON                1239
HYDERABAD              1029
NAVI MUMBAI - NMT      1025
NOIDA                   969
SOUTH MUMBAI            803
MUMBAI 2                678
CHENNAI                 564
KOLKATA                 334
JAIPUR                  199
NAGPUR                  160
OTHER CITY               70
KOCHI                    39
ZIRAKPUR                  6
LUDHIANA                  2
MOHALI                    2
PANCHKULA                 1
CHANDIGARH                1
DEHRADUN                  1
LUCKNOW                   1
DERABASSI                 1
Name: count, dtype: int64

In [ ]:
# society_master['City'] = society_master['City'].str.lower()
society_master['City'] = society_master['City'].replace('PUNE_PMC','Pune')
society_master['City'] = society_master['City'].replace('MUMBAI 2','South Mumbai')
society_master['City'] = society_master['City'].replace('NAVI MUMBAI - THANE','Navi Mumbai')
society_master['City'] = society_master['City'].replace('Navi Mumbai - Thane','Navi Mumbai')
society_master['City'] = society_master['City'].replace('NAVI MUMBAI - NMT','Navi Mumbai')
society_master['City'] = society_master['City'].replace('BANGALORE 1','Bangalore')
society_master['City'] = society_master['City'].replace('BANGALORE 2','Bangalore')
society_master['City'] = society_master['City'].replace('PUNE_PCMC','Pune')
society_master['City'] = society_master['City'].replace('WEST DELHI','Gurgaon')
society_master['City'] = society_master['City'].str.title()
society_master.rename(columns = {'City':'Location'},inplace=True)


In [ ]:
society_master['Location'].value_counts()

Location
Pune            3215
Navi Mumbai     2562
Mumbai          2151
Bangalore       2083
South Mumbai    1481
Ahmedabad       1265
Gurgaon         1239
Hyderabad       1029
Noida            969
Chennai          564
Kolkata          334
Jaipur           199
Nagpur           160
Other City        70
Kochi             39
Zirakpur           6
Ludhiana           2
Mohali             2
Panchkula          1
Chandigarh         1
Dehradun           1
Lucknow            1
Derabassi          1
Name: count, dtype: int64

In [ ]:
vms_onboarded = society_master[(society_master['VMSOnbDate']!="")]
vms_onboarded['VMSOnbDate'] = pd.to_datetime(vms_onboarded['VMSOnbDate'])
vms_onboarded = vms_onboarded[(vms_onboarded['VMSOnbDate'].dt.month==month) & (vms_onboarded['VMSOnbDate'].dt.year==year)]
vms_onboarded['Sold Flats'] = pd.to_numeric(vms_onboarded['Sold Flats'])

erp_onboarded = society_master[(society_master['ERPOnbDate']!="")]
#erp_onboarded['ERPOnbDate'] = pd.to_datetime(erp_onboarded['ERPOnbDate'])
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%d-%B-%Y')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%d-%b-%Y')
        except ValueError:
            return pd.to_datetime(date_str, format='%d/%m/%Y')

erp_onboarded['ERPOnbDate'] = erp_onboarded['ERPOnbDate'].apply(parse_date)

erp_onboarded = erp_onboarded[(erp_onboarded['ERPOnbDate'].dt.month==month) & (erp_onboarded['ERPOnbDate'].dt.year==year)]
erp_onboarded['Sold Flats'] = pd.to_numeric(erp_onboarded['Sold Flats'])


# both_onboarded_flats = both_onboarded[erp_onboarded['HO Product (ERP)']!='ERP ONLY']
# both_onboarded_flats = both_onboarded_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
# review = pd.merge(review,both_onboarded_flats,on='Location',how='left')
# review.rename(columns = {'Sold Flats':'Onboarded Flats (ERP Only)'},inplace=True)
review

<ipython-input-186-8f4e59309f7c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vms_onboarded['VMSOnbDate'] = pd.to_datetime(vms_onboarded['VMSOnbDate'])
<ipython-input-186-8f4e59309f7c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erp_onboarded['ERPOnbDate'] = erp_onboarded['ERPOnbDate'].apply(parse_date)
<ipython-input-186-8f4e59309f7c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,Location,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP,New+Winback Builders,Winback Flats,Upsell Flats,Budgeted HC (TSM+AASM)
0,Mumbai,650000,284946.76,19.0,3.0,0.0,1,1.0,1772,1725,439.0,1772,1333,47.0,392.0,0.0,350.0,0.0,NaN
1,Navi Mumbai,1000000,214938.87,9.0,1.0,0.0,1,1.0,2613,1710,1219.0,1768,1394,58.0,316.0,701.0,77.0,845.0,NaN
2,Hyderabad,1750000,1561572.541,11.0,2.0,1.0,0,1.0,4434,3765,2579.0,4226,1855,461.0,1910.0,678.0,0.0,208.0,NaN
3,Gurgaon,1250000,1061524.576,13.0,0.0,0.0,1,1.0,6364,1402,5010.0,3532,1301,2183.0,48.0,2119.0,585.0,2832.0,NaN
4,Pune,2250000,1559821.57,20.0,3.0,0.0,1,1.0,10614,9289,1876.0,9559,8698,310.0,551.0,393.0,2487.0,1055.0,NaN
5,Noida,2000000,1505727.421,13.0,1.0,0.0,1,1.0,4075,4075,2472.0,4075,1603,0.0,2472.0,1474.0,755.0,0.0,NaN
6,Ahmedabad,850000,545007.187,6.0,0.0,1.0,1,2.0,3378,3378,0.0,3378,3378,0.0,0.0,0.0,200.0,0.0,NaN
7,Chennai,350000,124184.215,4.0,0.0,0.0,1,1.0,1343,645,878.0,765,465,120.0,180.0,0.0,216.0,578.0,NaN
8,Bangalore,2500000,1127528.764,15.0,3.0,1.0,1,2.0,4702,3483,3541.0,3703,1161,220.0,2322.0,925.0,152.0,999.0,NaN
9,Kolkata,300000,341405.755,1.0,0.0,0.0,1,1.0,2054,1572,482.0,1572,1572,0.0,0.0,0.0,0.0,482.0,NaN


In [ ]:
vmsonly_onboarded_flats = vms_onboarded[vms_onboarded['HO Product (VMS)'].isin(['VMS ONLY','BOTH VMS + ERP'])]
vms_onboarded_flats = vmsonly_onboarded_flats[['Sold Flats','Location']].groupby('Location').sum().reset_index()

# vmsonly_onboarded_flats = vmsonly_onboarded_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
# review = pd.merge(review,vmsonly_onboarded_flats,on='Location',how='left')
# review.rename(columns = {'Sold Flats':'Onboarded Flats (VMS Only)'},inplace=True)

erponly_onboarded_flats = erp_onboarded[erp_onboarded['HO Product (ERP)'].isin(['ERP ONLY','BOTH VMS + ERP'])]
erp_onboarded_flats = erponly_onboarded_flats[['Sold Flats','Location']].groupby('Location').sum().reset_index()

vms_onboarded_flats.rename(columns = {'Sold Flats':'VMS Onboarded Flats'},inplace=True)
erp_onboarded_flats.rename(columns = {'Sold Flats':'ERP Onboarded Flats'},inplace=True)

In [ ]:
review = pd.merge(vms_onboarded_flats,review,on='Location',how='outer')
review = pd.merge(erp_onboarded_flats,review,on='Location',how='outer')
review = review[review['Location']!='Other City']
review

,Location,ERP Onboarded Flats,VMS Onboarded Flats,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,...,VMS Sold flats,ERP Sold flats,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP,New+Winback Builders,Winback Flats,Upsell Flats,Budgeted HC (TSM+AASM)
0,Ahmedabad,1332,1244,850000,545007.187,6.0,0.0,1.0,1.0,2.0,...,3378.0,0.0,3378.0,3378.0,0.0,0.0,0.0,200.0,0.0,NaN
1,Bangalore,2685,3966,2500000,1127528.764,15.0,3.0,1.0,1.0,2.0,...,3483.0,3541.0,3703.0,1161.0,220.0,2322.0,925.0,152.0,999.0,NaN
2,Chennai,176,689,350000,124184.215,4.0,0.0,0.0,1.0,1.0,...,645.0,878.0,765.0,465.0,120.0,180.0,0.0,216.0,578.0,NaN
3,Gurgaon,2108,140,1250000,1061524.576,13.0,0.0,0.0,1.0,1.0,...,1402.0,5010.0,3532.0,1301.0,2183.0,48.0,2119.0,585.0,2832.0,NaN
4,Hyderabad,632,2877,1750000,1561572.541,11.0,2.0,1.0,0.0,1.0,...,3765.0,2579.0,4226.0,1855.0,461.0,1910.0,678.0,0.0,208.0,NaN
5,Kolkata,577,966,300000,341405.755,1.0,0.0,0.0,1.0,1.0,...,1572.0,482.0,1572.0,1572.0,0.0,0.0,0.0,0.0,482.0,NaN
6,Mumbai,232,2811,650000,284946.76,19.0,3.0,0.0,1.0,1.0,...,1725.0,439.0,1772.0,1333.0,47.0,392.0,0.0,350.0,0.0,NaN
7,Navi Mumbai,811,3313,1000000,214938.87,9.0,1.0,0.0,1.0,1.0,...,1710.0,1219.0,1768.0,1394.0,58.0,316.0,701.0,77.0,845.0,NaN
8,Noida,4985,8884,2000000,1505727.421,13.0,1.0,0.0,1.0,1.0,...,4075.0,2472.0,4075.0,1603.0,0.0,2472.0,1474.0,755.0,0.0,NaN
10,Pune,3417,10487,2250000,1559821.57,20.0,3.0,0.0,1.0,1.0,...,9289.0,1876.0,9559.0,8698.0,310.0,551.0,393.0,2487.0,1055.0,NaN


In [ ]:
review['VMS Onbaording %'] = review['VMS Onboarded Flats']/review['VMS Sold flats']
review['ERP Onbaording %'] = review['ERP Onboarded Flats']/review['ERP Sold flats']
review['VMS Onbaording %'] = pd.to_numeric(review['VMS Onbaording %'])
review['VMS Onbaording %'] = review['VMS Onbaording %'].round(2).apply(lambda x: str(x*100)+'%')
review['ERP Onbaording %'] = review['ERP Onbaording %'].round(2).apply(lambda x: str(x*100)+'%')


In [ ]:
review

,Location,ERP Onboarded Flats,VMS Onboarded Flats,Target,Total HO IP,TSMs + AASMs at start of the month (on Field),ASMs at start of the month,DCHs at start of the month,CH Count,CHs/DCHs at start of the month,...,New+Winback Flats,New+Winback VMS Only,New+Winback ERP Only,New+Winback Both VMS + ERP,New+Winback Builders,Winback Flats,Upsell Flats,Budgeted HC (TSM+AASM),VMS Onbaording %,ERP Onbaording %
0,Ahmedabad,1332,1244,850000,545007.187,6.0,0.0,1.0,1.0,2.0,...,3378.0,3378.0,0.0,0.0,0.0,200.0,0.0,NaN,37.0%,inf%
1,Bangalore,2685,3966,2500000,1127528.764,15.0,3.0,1.0,1.0,2.0,...,3703.0,1161.0,220.0,2322.0,925.0,152.0,999.0,NaN,113.99999999999999%,76.0%
2,Chennai,176,689,350000,124184.215,4.0,0.0,0.0,1.0,1.0,...,765.0,465.0,120.0,180.0,0.0,216.0,578.0,NaN,107.0%,20.0%
3,Gurgaon,2108,140,1250000,1061524.576,13.0,0.0,0.0,1.0,1.0,...,3532.0,1301.0,2183.0,48.0,2119.0,585.0,2832.0,NaN,10.0%,42.0%
4,Hyderabad,632,2877,1750000,1561572.541,11.0,2.0,1.0,0.0,1.0,...,4226.0,1855.0,461.0,1910.0,678.0,0.0,208.0,NaN,76.0%,25.0%
5,Kolkata,577,966,300000,341405.755,1.0,0.0,0.0,1.0,1.0,...,1572.0,1572.0,0.0,0.0,0.0,0.0,482.0,NaN,61.0%,120.0%
6,Mumbai,232,2811,650000,284946.76,19.0,3.0,0.0,1.0,1.0,...,1772.0,1333.0,47.0,392.0,0.0,350.0,0.0,NaN,163.0%,53.0%
7,Navi Mumbai,811,3313,1000000,214938.87,9.0,1.0,0.0,1.0,1.0,...,1768.0,1394.0,58.0,316.0,701.0,77.0,845.0,NaN,194.0%,67.0%
8,Noida,4985,8884,2000000,1505727.421,13.0,1.0,0.0,1.0,1.0,...,4075.0,1603.0,0.0,2472.0,1474.0,755.0,0.0,NaN,218.00000000000003%,202.0%
10,Pune,3417,10487,2250000,1559821.57,20.0,3.0,0.0,1.0,1.0,...,9559.0,8698.0,310.0,551.0,393.0,2487.0,1055.0,NaN,112.99999999999999%,182.0%


In [ ]:
review['Productivity New+Winback (flats)'] = review['New+Winback Flats']/review['TSMs + AASMs at start of the month (on Field)']
review['Productivity New+Winback (Including ASM)'] = review['New+Winback Flats']/(review['TSMs + AASMs at start of the month (on Field)']+review['ASMs at start of the month'])
review['Overall Productivity'] = review['Total flats']/(review['TSMs + AASMs at start of the month (on Field)']+review['ASMs at start of the month'])
review['Winback%'] = review['Winback Flats']/review['Total flats']
review['Upsell%'] = review['Upsell Flats']/review['Total flats']
review['Winback%'] =review['Winback%'].round(2).apply(lambda x: str(x*100)+'%')
review['Upsell%'] = review['Upsell%'].round(2).apply(lambda x: str(x*100)+'%')
review.columns

Index(['Location', 'ERP Onboarded Flats', 'VMS Onboarded Flats', 'Target',
       'Total HO IP', 'TSMs + AASMs at start of the month (on Field)',
       'ASMs at start of the month', 'DCHs at start of the month', 'CH Count',
       'CHs/DCHs at start of the month', 'Total flats', 'VMS Sold flats',
       'ERP Sold flats', 'New+Winback Flats', 'New+Winback VMS Only',
       'New+Winback ERP Only', 'New+Winback Both VMS + ERP',
       'New+Winback Builders', 'Winback Flats', 'Upsell Flats',
       'Budgeted HC (TSM+AASM)', 'VMS Onbaording %', 'ERP Onbaording %',
       'Productivity New+Winback (flats)',
       'Productivity New+Winback (Including ASM)', 'Overall Productivity',
       'Winback%', 'Upsell%'],
      dtype='object')

In [ ]:
review = review[['Location','Budgeted HC (TSM+AASM)','TSMs + AASMs at start of the month (on Field)','ASMs at start of the month','CHs/DCHs at start of the month','Total flats',
                 'VMS Sold flats','ERP Sold flats','VMS Onboarded Flats','ERP Onboarded Flats','VMS Onbaording %','ERP Onbaording %','New+Winback VMS Only', 'New+Winback ERP Only',
                 'New+Winback Both VMS + ERP','New+Winback Builders','New+Winback Flats','Productivity New+Winback (flats)','Productivity New+Winback (Including ASM)',
                 'Overall Productivity','Winback Flats','Winback%','Upsell Flats','Upsell%']]

In [ ]:
import seaborn as sns

In [ ]:
cm = sns.light_palette("blue", as_cmap=True)
review.style.background_gradient(cmap=cm)
review

,Location,Budgeted HC (TSM+AASM),TSMs + AASMs at start of the month (on Field),ASMs at start of the month,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,VMS Onboarded Flats,ERP Onboarded Flats,...,New+Winback Both VMS + ERP,New+Winback Builders,New+Winback Flats,Productivity New+Winback (flats),Productivity New+Winback (Including ASM),Overall Productivity,Winback Flats,Winback%,Upsell Flats,Upsell%
0,Ahmedabad,NaN,6.0,0.0,2.0,3378.0,3378.0,0.0,1244,1332,...,0.0,0.0,3378.0,563.000000,563.000000,563.000000,200.0,6.0%,0.0,0.0%
1,Bangalore,NaN,15.0,3.0,2.0,4702.0,3483.0,3541.0,3966,2685,...,2322.0,925.0,3703.0,246.866667,205.722222,261.222222,152.0,3.0%,999.0,21.0%
2,Chennai,NaN,4.0,0.0,1.0,1343.0,645.0,878.0,689,176,...,180.0,0.0,765.0,191.250000,191.250000,335.750000,216.0,16.0%,578.0,43.0%
3,Gurgaon,NaN,13.0,0.0,1.0,6364.0,1402.0,5010.0,140,2108,...,48.0,2119.0,3532.0,271.692308,271.692308,489.538462,585.0,9.0%,2832.0,45.0%
4,Hyderabad,NaN,11.0,2.0,1.0,4434.0,3765.0,2579.0,2877,632,...,1910.0,678.0,4226.0,384.181818,325.076923,341.076923,0.0,0.0%,208.0,5.0%
5,Kolkata,NaN,1.0,0.0,1.0,2054.0,1572.0,482.0,966,577,...,0.0,0.0,1572.0,1572.000000,1572.000000,2054.000000,0.0,0.0%,482.0,23.0%
6,Mumbai,NaN,19.0,3.0,1.0,1772.0,1725.0,439.0,2811,232,...,392.0,0.0,1772.0,93.263158,80.545455,80.545455,350.0,20.0%,0.0,0.0%
7,Navi Mumbai,NaN,9.0,1.0,1.0,2613.0,1710.0,1219.0,3313,811,...,316.0,701.0,1768.0,196.444444,176.800000,261.300000,77.0,3.0%,845.0,32.0%
8,Noida,NaN,13.0,1.0,1.0,4075.0,4075.0,2472.0,8884,4985,...,2472.0,1474.0,4075.0,313.461538,291.071429,291.071429,755.0,19.0%,0.0,0.0%
10,Pune,NaN,20.0,3.0,1.0,10614.0,9289.0,1876.0,10487,3417,...,551.0,393.0,9559.0,477.950000,415.608696,461.478261,2487.0,23.0%,1055.0,10.0%


In [ ]:
# from gspread_dataframe import set_with_dataframe
# # from gspread_formatting import *
# spreadSheetId = "1ZwkFWS6ho0VGYYmbV5309BA4eRHDKrQ53NwoZgkZBeI"
# rangeName = "Sheet1"
# wb = gc.open_by_key(spreadSheetId)
# ws = wb.worksheet(rangeName)
# ws.clear()

# # df = pd.DataFrame(mc_election_final)
# set_with_dataframe(ws, review.sort_values('Budgeted HC (TSM+AASM)',ascending = False))

#**Target vs Achievment**

In [ ]:
# importing from DS-Sales Dashboard sheet
# key = "1fTX5qhmRjQTdqYHQCxgCJ5RyLkN93HCoGqfKGjIgVbA"
# range = "City Target"
# wb = gc.open_by_key(key)
# ws = wb.worksheet(range)
# citytarget = get_data_df(key, range)

# importing from DS-Sales Dashboard RP sheet
key = "1uptLp8OD8G9HaojJLeCG1KaZWGrIqT3WAamqFX_a-MM"
range = "City Target"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
citytarget = get_data_df(key, range)
citytarget.rename({'RP Target':'Target','RP_HO':'HO IP'},inplace=True,axis=1)

In [ ]:
citytarget.columns

Index(['Month', 'Year', 'City Type', 'City', 'SL', 'SLName', 'Total Closure',
       'Sold Flats', 'New/Winback Clsoure', 'Upsell Closures',
       'New/Winback Flats', 'Upsell Flats', 'New/Winback Flats (HO+ve)',
       'Target', 'RP_D4', 'HO IP', 'RP_D1', 'Revenue Target',
       'Revenue Achievement', 'RP D4 Ach%', 'RP HO Ach%', 'RP D1 Ach%',
       'Revenue Ach%', 'D4 Ach%', 'HO Ach%', 'D1 Ach%',
       'Gross RP without Offset'],
      dtype='object')

In [ ]:
citytarget = citytarget[(citytarget['Month']==str(month)) & (citytarget['Year']==str(year))]
citytarget = citytarget[['City','Target','Total Closure','HO IP','HO Ach%','Month','Year']]

citytarget = citytarget.rename(columns = {'City':'Location'})

In [ ]:
citytarget

,Location,Target,Total Closure,HO IP,HO Ach%,Month,Year
53,Pune,2250000,62,1559822,95.94%,3,2024
54,Bangalore,2500000,37,1127529,48.02%,3,2024
55,Noida,2000000,16,1505727,66.55%,3,2024
56,Ahmedabad,850000,15,545007,63.53%,3,2024
57,Hyderabad,1750000,17,1561573,87.70%,3,2024
58,Gurgaon,1250000,17,1061525,60.72%,3,2024
59,Navi Mumbai,1000000,13,214939,34.32%,3,2024
60,Mumbai,650000,23,284947,56.06%,3,2024
61,Chennai,350000,6,124184,25.46%,3,2024
62,Kolkata,300000,9,341406,103.93%,3,2024


In [ ]:
#Payout calculation sheet for TSM & ASM Count
key = "1EcV-4vhUf9hpMEWP1hpo6Jl3FU8O3VEFani7l47v8VM"
range = "TSM"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
tsm = get_data_df(key, range)

key = "1EcV-4vhUf9hpMEWP1hpo6Jl3FU8O3VEFani7l47v8VM"
range = "AASM"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
aasm = get_data_df(key, range)

tsm = tsm[(tsm['Month']==str(month)) & (tsm['Year']==str(year))]
tsm_count = tsm[['Emp City','EMP Code']].groupby('Emp City').count().reset_index()
tsm_count = tsm_count.rename(columns = {'Emp City':'Location','EMP Code':'TSM Count'})

tsm_central = tsm[tsm['Team']=='Central']
tsm_central = tsm_central[['Emp City','EMP Code']].groupby('Emp City').count().reset_index()
tsm_central = tsm_central.rename(columns = {'Emp City':'Location','EMP Code':'Dedicated Central TSMs'})

tsm_erp = tsm[tsm['Team'].isin(['ERP','ERP team'])]
tsm_erp = tsm_erp[['Emp City','EMP Code']].groupby('Emp City').count().reset_index()
tsm_erp = tsm_erp.rename(columns = {'Emp City':'Location','EMP Code':'Dedicated ERP TSMs'})

aasm = aasm[(aasm['Month']==str(month)) & (aasm['Year']==str(year))]
aasm_count = aasm[['Emp City','EMP Code']].groupby('Emp City').count().reset_index()
aasm_count = aasm_count.rename(columns = {'Emp City':'Location','EMP Code':'AASM Count'})

tsm['Amount (HO)'] = pd.to_numeric(tsm['Amount (HO)'])
tsm_earners = tsm[tsm['Amount (HO)']>0]
tsm_earners = tsm_earners[['Emp City','EMP Code']].groupby('Emp City').count().reset_index()
tsm_earners = tsm_earners.rename(columns = {'Emp City':'Location','EMP Code':'TSM Count (Earners)'})

aasm['Amount (HO)'] = pd.to_numeric(aasm['Amount (HO)'])
aasm_earners = aasm[aasm['Amount (HO)']>0]
aasm_earners = aasm_earners[['Emp City','EMP Code']].groupby('Emp City').count().reset_index()
aasm_earners.rename(columns = {'Emp City':'Location','EMP Code':'AASM Count (Earners)'},inplace=True)

<ipython-input-200-7c29f3c3da35>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsm['Amount (HO)'] = pd.to_numeric(tsm['Amount (HO)'])


In [ ]:
review1 = pd.merge(review,pd.merge(citytarget,tsm_count,how='outer',on='Location'),how='outer',on='Location')
review1  = pd.merge(review1,aasm_count,how='outer',on='Location')
review1 =  pd.merge(review1,pd.merge(tsm_earners,aasm_earners,how='outer',on='Location'),how='outer',on='Location')
review1 =  pd.merge(review1,pd.merge(tsm_erp,tsm_central,how='outer',on='Location'),how='outer',on='Location').fillna(0)

In [ ]:
review1['HO IP'] = pd.to_numeric(review1['HO IP'])
# review1['Sold Flats'] = pd.to_numeric(review1['Sold Flats'])
review1['Target'] = review1['Target'].apply(lambda x: str(x).replace(',',''))
review1['Target'] = pd.to_numeric(review1['Target'])

review1['IP/FC'] = review1['HO IP']/review1['Total flats']
review1['IP Target/TSM'] = review1['Target']/review1['TSM Count']
review1['IP Achievement/TSM'] = review1['HO IP']/review1['TSM Count']
review1['Number of Earners (TSM+AASM)'] = review1['TSM Count (Earners)']+review1['AASM Count (Earners)']
review1['% of Earners'] = review1['Number of Earners (TSM+AASM)']/(review1['TSM Count']+review1['AASM Count'])
review1[['% of Earners']] = review1[['% of Earners']].apply(percentage)

In [ ]:
review1

,Location,Budgeted HC (TSM+AASM),TSMs + AASMs at start of the month (on Field),ASMs at start of the month,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,VMS Onboarded Flats,ERP Onboarded Flats,...,AASM Count,TSM Count (Earners),AASM Count (Earners),Dedicated ERP TSMs,Dedicated Central TSMs,IP/FC,IP Target/TSM,IP Achievement/TSM,Number of Earners (TSM+AASM),% of Earners
0,Ahmedabad,0,6.0,0.0,2.0,3378.0,3378.0,0.0,1244,1332,...,1.0,4,1.0,0.0,0.0,161.340142,170000.000000,109001.400000,5.0,83.33%
1,Bangalore,0,15.0,3.0,2.0,4702.0,3483.0,3541.0,3966,2685,...,3.0,7,3.0,0.0,0.0,239.797746,227272.727273,102502.636364,10.0,71.43%
2,Chennai,0,4.0,0.0,1.0,1343.0,645.0,878.0,689,176,...,1.0,2,0.0,0.0,0.0,92.467610,116666.666667,41394.666667,2.0,50.0%
3,Gurgaon,0,13.0,0.0,1.0,6364.0,1402.0,5010.0,140,2108,...,2.0,5,1.0,0.0,7.0,166.801540,138888.888889,117947.222222,6.0,54.55%
4,Hyderabad,0,11.0,2.0,1.0,4434.0,3765.0,2579.0,2877,632,...,1.0,3,1.0,0.0,3.0,352.181552,194444.444444,173508.111111,4.0,40.0%
5,Kolkata,0,1.0,0.0,1.0,2054.0,1572.0,482.0,966,577,...,0.0,1,0.0,0.0,1.0,166.215190,300000.000000,341406.000000,1.0,100.0%
6,Mumbai,0,19.0,3.0,1.0,1772.0,1725.0,439.0,2811,232,...,6.0,9,4.0,0.0,1.0,160.805305,50000.000000,21919.000000,13.0,68.42%
7,Navi Mumbai,0,9.0,1.0,1.0,2613.0,1710.0,1219.0,3313,811,...,2.0,1,1.0,0.0,2.0,82.257558,142857.142857,30705.571429,2.0,22.22%
8,Noida,0,13.0,1.0,1.0,4075.0,4075.0,2472.0,8884,4985,...,2.0,8,1.0,0.0,3.0,369.503558,222222.222222,167303.000000,9.0,81.82%
9,Pune,0,20.0,3.0,1.0,10614.0,9289.0,1876.0,10487,3417,...,3.0,14,2.0,0.0,4.0,146.958922,132352.941176,91754.235294,16.0,80.0%


#**Society Count**


In [ ]:
new_win_societies = salesmaster[salesmaster['Closure Type'].isin(['New Closure','Winback'])]
new_win_soc = new_win_societies[['Location','Sold Flats']].groupby('Location').count().reset_index()
review2 = pd.merge(review1,new_win_soc,on='Location',how='outer')
review2.rename(columns = {'Sold Flats':'New+Winback Societies'},inplace=True)

total_societies = salesmaster[['Location','Sold Flats']].groupby('Location').count().reset_index() #Need to be changed
review2 = pd.merge(review2,total_societies,on='Location',how='inner')
review2.rename(columns = {'Sold Flats':'Total societies'},inplace=True)

review2['FC/Society (New+Winback)'] = review2['New+Winback Flats']/review2['New+Winback Societies']
# review2['Productivity (Societies)'] = review2['New+Winback Flats']/review2['New+Winback Societies']
review2['Productivity (Society including upsell)'] = review2['Total flats']/review2['Total societies']

#**Flat Counts**


In [ ]:
vmsonly_flats = salesmaster[salesmaster['HO Product'].isin(['VMS only'])]
vmsonly_flats = vmsonly_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review3 = pd.merge(review2,vmsonly_flats,on='Location',how='outer')
review3.rename(columns = {'Sold Flats':'VMS Only Sold flats'},inplace=True)

erponly_flats = salesmaster[salesmaster['HO Product'].isin(['ERP only'])]
erponly_flats = erponly_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review3 = pd.merge(review3,erponly_flats,on='Location',how='outer')
review3.rename(columns = {'Sold Flats':'ERP Only Sold flats'},inplace=True)

vms_erp_flats = salesmaster[salesmaster['HO Product'].isin(['Both VMS + ERP'])]
vms_erp_flats = vms_erp_flats[['Location','Sold Flats']].groupby('Location').sum().reset_index()
review3 = pd.merge(review3,vms_erp_flats,on='Location',how='outer')
review3.rename(columns = {'Sold Flats':'VMS + ERP Sold flats'},inplace=True)


In [ ]:
review3 = review3.fillna(0)
review3['VMS %'] = review3['VMS Only Sold flats']/review3['Total flats']
review3['ERP %'] = review3['ERP Only Sold flats']/review3['Total flats']
review3['VMS + ERP %'] = review3['VMS + ERP Sold flats']/review3['Total flats']

review3['VMS %']  = review3['VMS %'].round(2).apply(lambda x: str(x*100)+'%')
review3['ERP %'] = review3['ERP %'].round(2).apply(lambda x: str(x*100)+'%')
review3['VMS + ERP %'] = review3['VMS + ERP %'].round(2).apply(lambda x: str(x*100)+'%')
review3

,Location,Budgeted HC (TSM+AASM),TSMs + AASMs at start of the month (on Field),ASMs at start of the month,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,VMS Onboarded Flats,ERP Onboarded Flats,...,New+Winback Societies,Total societies,FC/Society (New+Winback),Productivity (Society including upsell),VMS Only Sold flats,ERP Only Sold flats,VMS + ERP Sold flats,VMS %,ERP %,VMS + ERP %
0,Ahmedabad,0,6.0,0.0,2.0,3378.0,3378.0,0.0,1244,1332,...,14,14,241.285714,241.285714,3378,0.0,0.0,100.0%,0.0%,0.0%
1,Bangalore,0,15.0,3.0,2.0,4702.0,3483.0,3541.0,3966,2685,...,27,33,137.148148,142.484848,1161,1219.0,2322.0,25.0%,26.0%,49.0%
2,Chennai,0,4.0,0.0,1.0,1343.0,645.0,878.0,689,176,...,4,6,191.250000,223.833333,465,698.0,180.0,35.0%,52.0%,13.0%
3,Gurgaon,0,13.0,0.0,1.0,6364.0,1402.0,5010.0,140,2108,...,6,16,588.666667,397.750000,1354,4962.0,48.0,21.0%,78.0%,1.0%
4,Hyderabad,0,11.0,2.0,1.0,4434.0,3765.0,2579.0,2877,632,...,12,14,352.166667,316.714286,1855,669.0,1910.0,42.0%,15.0%,43.0%
5,Kolkata,0,1.0,0.0,1.0,2054.0,1572.0,482.0,966,577,...,7,9,224.571429,228.222222,1572,482.0,0.0,77.0%,23.0%,0.0%
6,Mumbai,0,19.0,3.0,1.0,1772.0,1725.0,439.0,2811,232,...,17,17,104.235294,104.235294,1333,47.0,392.0,75.0%,3.0%,22.0%
7,Navi Mumbai,0,9.0,1.0,1.0,2613.0,1710.0,1219.0,3313,811,...,9,12,196.444444,217.750000,1394,903.0,316.0,53.0%,35.0%,12.0%
8,Noida,0,13.0,1.0,1.0,4075.0,4075.0,2472.0,8884,4985,...,10,13,407.500000,313.461538,1603,0.0,2472.0,39.0%,0.0%,61.0%
9,Pune,0,20.0,3.0,1.0,10614.0,9289.0,1876.0,10487,3417,...,40,52,238.975000,204.115385,8738,1325.0,551.0,82.0%,12.0%,5.0%


#**DEMO Data**

In [ ]:
# importing MasterDemo sheet
key = "1TGX2XuuzC_kZD_H30fdqWxD4GXiVdYY6HUdXU2lOS4c"
range = "masterDEMO"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
demo = get_data_df(key, range)

demo['Meeting Start Date'] = pd.to_datetime(demo['Meeting Start Date'])
demo = demo[(demo['Meeting Start Date'].dt.month == month) & (demo['Meeting Start Date'].dt.year == year)]
demo.shape

(1972, 43)

In [ ]:
demo_tsm = demo[demo['Role'].isin(['TSM','Senior TSM','Sales Executive'])]
demo_citywise = demo[['City','Emp Code']].groupby('City').count().reset_index()
demo_citywise.rename(columns = {'City':'Location','Emp Code':'Demo Volume'},inplace=True)
demo_citywise = demo_citywise.drop(0)

In [ ]:
review4 = pd.merge(review3,demo_citywise,on='Location',how='left').fillna(0)
review4['Demo/TSM'] = review4['Demo Volume']/review4['TSMs + AASMs at start of the month (on Field)']

In [ ]:
review4

,Location,Budgeted HC (TSM+AASM),TSMs + AASMs at start of the month (on Field),ASMs at start of the month,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,VMS Onboarded Flats,ERP Onboarded Flats,...,FC/Society (New+Winback),Productivity (Society including upsell),VMS Only Sold flats,ERP Only Sold flats,VMS + ERP Sold flats,VMS %,ERP %,VMS + ERP %,Demo Volume,Demo/TSM
0,Ahmedabad,0,6.0,0.0,2.0,3378.0,3378.0,0.0,1244,1332,...,241.285714,241.285714,3378,0.0,0.0,100.0%,0.0%,0.0%,114,19.000000
1,Bangalore,0,15.0,3.0,2.0,4702.0,3483.0,3541.0,3966,2685,...,137.148148,142.484848,1161,1219.0,2322.0,25.0%,26.0%,49.0%,412,27.466667
2,Chennai,0,4.0,0.0,1.0,1343.0,645.0,878.0,689,176,...,191.250000,223.833333,465,698.0,180.0,35.0%,52.0%,13.0%,13,3.250000
3,Gurgaon,0,13.0,0.0,1.0,6364.0,1402.0,5010.0,140,2108,...,588.666667,397.750000,1354,4962.0,48.0,21.0%,78.0%,1.0%,43,3.307692
4,Hyderabad,0,11.0,2.0,1.0,4434.0,3765.0,2579.0,2877,632,...,352.166667,316.714286,1855,669.0,1910.0,42.0%,15.0%,43.0%,140,12.727273
5,Kolkata,0,1.0,0.0,1.0,2054.0,1572.0,482.0,966,577,...,224.571429,228.222222,1572,482.0,0.0,77.0%,23.0%,0.0%,30,30.000000
6,Mumbai,0,19.0,3.0,1.0,1772.0,1725.0,439.0,2811,232,...,104.235294,104.235294,1333,47.0,392.0,75.0%,3.0%,22.0%,359,18.894737
7,Navi Mumbai,0,9.0,1.0,1.0,2613.0,1710.0,1219.0,3313,811,...,196.444444,217.750000,1394,903.0,316.0,53.0%,35.0%,12.0%,204,22.666667
8,Noida,0,13.0,1.0,1.0,4075.0,4075.0,2472.0,8884,4985,...,407.500000,313.461538,1603,0.0,2472.0,39.0%,0.0%,61.0%,164,12.615385
9,Pune,0,20.0,3.0,1.0,10614.0,9289.0,1876.0,10487,3417,...,238.975000,204.115385,8738,1325.0,551.0,82.0%,12.0%,5.0%,378,18.900000


#**Distance Travelled**


In [ ]:
key = "1-OGpSnP-Ok5FgQ7G2Rn_1OTqGal5EcI-xfuGD1BV9wo"
range = "Distance_Group"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
distance = get_data_df(key, range)

distance['Date'] = pd.to_datetime(distance['Date'])
distance = distance[(distance['Date'].dt.month == month) & (distance['Date'].dt.year == year)]
distance['total_distance'] = pd.to_numeric(distance['total_distance'])
distance = distance[['City','total_distance']].groupby('City').sum().reset_index()
distance.rename(columns = {'City':'Location','total_distance':'Total Distance'},inplace=True)

review5 = pd.merge(review4,distance,on='Location',how='left').fillna(0)
review5


,Location,Budgeted HC (TSM+AASM),TSMs + AASMs at start of the month (on Field),ASMs at start of the month,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,VMS Onboarded Flats,ERP Onboarded Flats,...,Productivity (Society including upsell),VMS Only Sold flats,ERP Only Sold flats,VMS + ERP Sold flats,VMS %,ERP %,VMS + ERP %,Demo Volume,Demo/TSM,Total Distance
0,Ahmedabad,0,6.0,0.0,2.0,3378.0,3378.0,0.0,1244,1332,...,241.285714,3378,0.0,0.0,100.0%,0.0%,0.0%,114,19.000000,3559.03
1,Bangalore,0,15.0,3.0,2.0,4702.0,3483.0,3541.0,3966,2685,...,142.484848,1161,1219.0,2322.0,25.0%,26.0%,49.0%,412,27.466667,4207.03
2,Chennai,0,4.0,0.0,1.0,1343.0,645.0,878.0,689,176,...,223.833333,465,698.0,180.0,35.0%,52.0%,13.0%,13,3.250000,966.82
3,Gurgaon,0,13.0,0.0,1.0,6364.0,1402.0,5010.0,140,2108,...,397.750000,1354,4962.0,48.0,21.0%,78.0%,1.0%,43,3.307692,1888.61
4,Hyderabad,0,11.0,2.0,1.0,4434.0,3765.0,2579.0,2877,632,...,316.714286,1855,669.0,1910.0,42.0%,15.0%,43.0%,140,12.727273,2056.04
5,Kolkata,0,1.0,0.0,1.0,2054.0,1572.0,482.0,966,577,...,228.222222,1572,482.0,0.0,77.0%,23.0%,0.0%,30,30.000000,509.07
6,Mumbai,0,19.0,3.0,1.0,1772.0,1725.0,439.0,2811,232,...,104.235294,1333,47.0,392.0,75.0%,3.0%,22.0%,359,18.894737,7698.38
7,Navi Mumbai,0,9.0,1.0,1.0,2613.0,1710.0,1219.0,3313,811,...,217.750000,1394,903.0,316.0,53.0%,35.0%,12.0%,204,22.666667,5002.48
8,Noida,0,13.0,1.0,1.0,4075.0,4075.0,2472.0,8884,4985,...,313.461538,1603,0.0,2472.0,39.0%,0.0%,61.0%,164,12.615385,3689.48
9,Pune,0,20.0,3.0,1.0,10614.0,9289.0,1876.0,10487,3417,...,204.115385,8738,1325.0,551.0,82.0%,12.0%,5.0%,378,18.900000,3429.24


#**Leadgen Data**

In [ ]:
# key = "18s8FgRpWCzcz-_rypptVeO850_3MhKvnhS95YFRXWkg"
key ="1P0M_w96xv5wlbdIvlBba5zHHtNAildloA8AEfCVqsqU"
range = "Sheet1"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
leadgen = get_data_df(key, range)

range = "Sheet2"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
closure = get_data_df(key, range)

closure.columns = closure.iloc[4]
closure = closure.loc[5:,:]

In [ ]:
leadgen['meeting fixed date'] = pd.to_datetime(leadgen['meeting fixed date'])
meeting_fixed = leadgen[(leadgen['meeting fixed date'].dt.month == month) & (leadgen['meeting fixed date'].dt.year == year)]
meeting_fixed['Checkbox the City'] = meeting_fixed['Checkbox the City'].replace('Thane','Navi Mumbai')
meeting_fixed['Checkbox the City'] = meeting_fixed['Checkbox the City'].replace('South Bombay','South Mumbai')
meeting_fixed_count = meeting_fixed[['Checkbox the City','Lead ID']].groupby('Checkbox the City').count().reset_index()
meeting_fixed_count.rename(columns={'Checkbox the City':'Location','Lead ID':'Meetings Fixed count'},inplace=True)
meeting_fixed['Number of Flats in the society'] = meeting_fixed['Number of Flats in the society'].apply(lambda x: x.replace(",",''))
meeting_fixed['Number of Flats in the society'] = pd.to_numeric(meeting_fixed['Number of Flats in the society'])
meeting_fixed_fc = meeting_fixed[['Checkbox the City','Number of Flats in the society']].groupby('Checkbox the City').sum().reset_index()
meeting_fixed_fc.rename(columns={'Checkbox the City':'Location','Number of Flats in the society':'Meetings Fixed (FC)'},inplace=True)

invalid = meeting_fixed[meeting_fixed['Lead Status'].isin(['closed_invalid'])]
invalid = invalid[['Checkbox the City','Lead ID']].groupby('Checkbox the City').count().reset_index()
invalid.rename(columns={'Checkbox the City':'Location','Lead ID':'Invalid Leads'},inplace=True)

valid = meeting_fixed[~(meeting_fixed['Lead Status'].isin(['closed_invalid']))]
valid = valid[['Checkbox the City','Lead ID']].groupby('Checkbox the City').count().reset_index()
valid.rename(columns={'Checkbox the City':'Location','Lead ID':'Valid Leads'},inplace=True)

valid_fc = meeting_fixed[~(meeting_fixed['Lead Status'].isin(['closed_invalid']))]
valid_fc = valid_fc[['Checkbox the City','Number of Flats in the society']].groupby('Checkbox the City').sum().reset_index()
valid_fc.rename(columns={'Checkbox the City':'Location','Number of Flats in the society':'Valid (FC)'},inplace=True)

lost = meeting_fixed[meeting_fixed['Lead Status'].isin(['closed_lost'])]
lost = lost[['Checkbox the City','Lead ID']].groupby('Checkbox the City').count().reset_index()
lost.rename(columns={'Checkbox the City':'Location','Lead ID':'Closed Lost (Count)'},inplace=True)

meeting_fixed['demo completed in same month'] = pd.to_numeric(meeting_fixed['demo completed in same month'])
meeting_fixed = meeting_fixed[~(meeting_fixed['Lead Status'].isin(['closed_invalid']))]
demo_completed = meeting_fixed[['Checkbox the City','demo completed in same month']].groupby('Checkbox the City').sum().reset_index()
demo_completed.rename(columns={'Checkbox the City':'Location','demo completed in same month':'Demo completed Leads'},inplace=True)

#meeting_fixed['Number of Flats in the society'] = meeting_fixed['Number of Flats in the society'].apply(lambda x: x.replace(',',''))
meeting_fixed['Number of Flats in the society']=pd.to_numeric(meeting_fixed['Number of Flats in the society'])
demo_completed_fc = meeting_fixed[meeting_fixed['demo completed in same month']>0]
demo_completed_fc = demo_completed_fc[['Checkbox the City','Number of Flats in the society']].groupby('Checkbox the City').sum().reset_index()
demo_completed_fc.rename(columns={'Checkbox the City':'Location','Number of Flats in the society':'Demo completed flats'},inplace=True)


<ipython-input-213-d5f521d36e0e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meeting_fixed['Checkbox the City'] = meeting_fixed['Checkbox the City'].replace('Thane','Navi Mumbai')
<ipython-input-213-d5f521d36e0e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meeting_fixed['Checkbox the City'] = meeting_fixed['Checkbox the City'].replace('South Bombay','South Mumbai')
<ipython-input-213-d5f521d36e0e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [ ]:
closure['closure date'] = pd.to_datetime(closure['closure date'])
closure = closure.replace('Thane','Navi Mumbai')
closure = closure.replace('South Bombay','South Mumbai')
closure = closure[(closure['closure date'].dt.month == month) & (closure['closure date'].dt.year == year)]
closure = closure[closure['lead type'].isin(['Central','Central - 3x Focus Leads'])]
closure['sold flats'] = pd.to_numeric(closure['sold flats'])
closure_fc = closure[['city map','sold flats']].groupby('city map').sum().reset_index()
closure_fc.rename(columns = {'city map':'Location','sold flats':'Central Closures (FC)'},inplace=True)
closure_fc = closure_fc[closure_fc['Location']!='']
closure_fc

4,Location,Central Closures (FC)
1,Ahmedabad,1104
2,Bangalore,3183
3,Chennai,1229
4,Gurgaon,3124
5,Hyderabad,1296
6,Jaipur,521
7,Kolkata,1080
8,Mumbai,1550
9,Nagpur,198
10,Navi Mumbai,3873


In [ ]:
leadgen_final = pd.merge(meeting_fixed_count,pd.merge(meeting_fixed_fc,invalid,how='outer',on='Location'),how='outer',on='Location')
leadgen_final = pd.merge(leadgen_final,pd.merge(valid,valid_fc,how='outer',on='Location'),how='outer',on='Location')
leadgen_final = pd.merge(leadgen_final,pd.merge(lost,demo_completed,how='outer',on='Location'),how='outer',on='Location')
leadgen_final = pd.merge(leadgen_final,pd.merge(demo_completed_fc,closure_fc,how='outer',on='Location'),how='outer',on='Location').fillna(0)

leadgen_final['Demo Conduction % (Leads)'] = leadgen_final['Demo completed Leads']/leadgen_final['Valid Leads']
leadgen_final['Demo Conduction % (FC)'] = leadgen_final['Demo completed flats']/leadgen_final['Valid (FC)']
leadgen_final['Demo Conduction % (Leads)'] =leadgen_final['Demo Conduction % (Leads)'].round(2).apply(lambda x: str(x*100)+'%')
leadgen_final['Demo Conduction % (FC)'] = leadgen_final['Demo Conduction % (FC)'].round(2).apply(lambda x: str(x*100)+'%')
leadgen_final

,Location,Meetings Fixed count,Meetings Fixed (FC),Invalid Leads,Valid Leads,Valid (FC),Closed Lost (Count),Demo completed Leads,Demo completed flats,Central Closures (FC),Demo Conduction % (Leads),Demo Conduction % (FC)
0,Ahmedabad,12.0,4253.0,9.0,3.0,970.0,0.0,3.0,970.0,1104.0,100.0%,100.0%
1,Bangalore,50.0,19588.0,7.0,43.0,14685.0,9.0,35.0,9977.0,3183.0,81.0%,68.0%
2,Chennai,15.0,8646.0,12.0,3.0,1533.0,0.0,2.0,853.0,1229.0,67.0%,56.00000000000001%
3,Gurgaon,19.0,13089.0,7.0,12.0,6739.0,2.0,7.0,3735.0,3124.0,57.99999999999999%,55.00000000000001%
4,Hyderabad,14.0,5158.0,7.0,7.0,1830.0,2.0,6.0,830.0,1296.0,86.0%,45.0%
5,Jaipur,3.0,3375.0,2.0,1.0,664.0,0.0,1.0,664.0,521.0,100.0%,100.0%
6,Kolkata,5.0,6596.0,0.0,5.0,6596.0,1.0,5.0,6596.0,1080.0,100.0%,100.0%
7,Mumbai,20.0,7750.0,11.0,9.0,2013.0,0.0,6.0,1618.0,1550.0,67.0%,80.0%
8,Navi Mumbai,30.0,10931.0,8.0,22.0,7950.0,9.0,20.0,6660.0,3873.0,91.0%,84.0%
9,Noida,24.0,22629.0,4.0,20.0,18712.0,5.0,17.0,16589.0,8114.0,85.0%,89.0%


In [ ]:
review6 = pd.merge(review5,leadgen_final,on='Location',how='left').fillna(0)
review6.shape

(10, 65)

#**Compliance**

In [ ]:
# ASM Demo Audit Compliance

key = "1w-XtlJlJm-hnoCJf7y5EbD6B88rvncy5TkE3uMj-8qo"
range = "Form Responses 1"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
demo_audit = get_data_df(key, range)

demo_audit['Timestamp'] = pd.to_datetime(demo_audit['Timestamp'],errors = 'coerce')
demo_audit = demo_audit[(demo_audit['Timestamp'].dt.month == month) & (demo_audit['Timestamp'].dt.year == year)]
demo_audit = demo_audit[['City','Enter Auditor name below']].groupby('City').count().reset_index()
demo_audit.rename(columns={'City':'Location','Enter Auditor name below':'ASM Demo Audits'},inplace=True)
demo_audit = demo_audit.replace('South Bombay','South Mumbai')

In [ ]:
demo_audit

,Location,ASM Demo Audits
0,Bengaluru,5
1,Hyderabad,13
2,Mumbai,28


In [ ]:
# Monthly Non Compliance

key = non_compliant_list[month] # changes with month
range = "Final"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
compliance = get_data_df(key, range)
compliance['Non-Compliant%'] = compliance['Non-Compliant%'].apply(lambda x: x.split('%')[0])
compliance['Non-Compliant%'] = pd.to_numeric(compliance['Non-Compliant%'])
compliance['Non-Compliant'] = pd.to_numeric(compliance['Non-Compliant'])
compliance = compliance[['Location','Non-Compliant','Non-Compliant%']].groupby('Location').agg({'Non-Compliant':'sum','Non-Compliant%':'mean'}).reset_index()

compliance['Non-Compliant%'] = compliance['Non-Compliant%'].apply(lambda x: str(x)+"%")


In [ ]:
review7 = pd.merge(review6,compliance,on='Location',how='left')
review8 = pd.merge(review7,demo_audit,on='Location',how='left').fillna(0)
review8.shape

(10, 68)

#**Election Data**

In [ ]:
key = "1Jbug_WbwnnuK32nT-UOq2fZPuJ9yxoGXvgVkIRSB3g4"
range = "mc_election_lead"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
election = get_data_df(key, range)

election['mc_election_date'] = pd.to_datetime(election['mc_election_date'])
election = election[(election['mc_election_date'].dt.month==month) & (election['mc_election_date'].dt.year==year)]
election['lead_city'] = election['lead_city'].replace('Thane','Navi Mumbai')
election['lead_city'] = election['lead_city'].replace('Navi_Mumbai','Navi Mumbai')

In [ ]:
election.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 5 to 182
Data columns (total 30 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   lead_id                                 70 non-null     object        
 1   last_updated_on                         70 non-null     object        
 2   mc_election_date                        70 non-null     datetime64[ns]
 3   lead_city                               70 non-null     object        
 4   lead_status                             70 non-null     object        
 5   Emp Code                                70 non-null     object        
 6   Official Email ID                       70 non-null     object        
 7   Role                                    70 non-null     object        
 8   resource_name                           70 non-null     object        
 9   created_by_resource_name                70 non-null     obje

In [ ]:
DumpDate = "30 July 2023"
file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Jul 2023/"
dump_name = 'MIS-Tab1-Prospect-A new 16 dec'
prospect = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)

prospect['lead_city'] = prospect['lead_city'].apply(lambda x: str(x).title())
prospect = prospect.replace({'lead_city':{'Navi Mumbai - Nmt':'Navi Mumbai','Navi Mumbai - Thane':'Navi Mumbai','Thane':'Navi Mumbai','Delhi_Central':'Gurgaon','Delhi':'Gurgaon',
                                                    'ghaziabad':'Noida','Ghaziabad':'Noida','Navi_Mumbai':'Navi Mumbai','South_Bombay':'South Mumbai','South_Bombay_2':'South Mumbai',
                                          'Mysore':'Bangalore','Karwar':'Bangalore','Pithoragarh':'Noida'}})
prospect['created_on'] = prospect['created_on'].apply(lambda x: x.split("T")[0])
prospect['created_on'] = pd.to_datetime(prospect['created_on'])
prospect = prospect[(prospect['created_on'].dt.month == month) & (prospect['created_on'].dt.year==year)]

total_leads = prospect[['lead_city','lead_id']].groupby('lead_city').count().reset_index()
total_leads.rename(columns={'lead_city':'Location','lead_id':'Leads created (Total)'},inplace=True)

election_lead = prospect[prospect['mc_election_date'].notnull()]
election_lead = election_lead[['lead_city','lead_id']].groupby('lead_city').count().reset_index()
election_lead.rename(columns={'lead_city':'Location','lead_id':'MC Election Lead'},inplace=True)


In [ ]:
prospect_data = pd.merge(total_leads,election_lead,on='Location',how='outer')
review9 = pd.merge(review8,prospect_data,on='Location',how='outer').fillna(0)
review9.shape

(10, 70)

In [ ]:
election_count = election[['lead_city','mc_election_date']].groupby('lead_city').count().reset_index()
election_count.rename(columns = {'lead_city':'Location','mc_election_date':'Election date in this month'},inplace=True)

post_election_count = election[election['MC Election Lead Status']=='Activity post MC Election Date']
post_election_count = post_election_count[['lead_city','mc_election_date']].groupby('lead_city').count().reset_index()
post_election_count.rename(columns = {'lead_city':'Location','mc_election_date':'Activity post election date (same month)'},inplace=True)

election['Demo date on or after Election date'] = pd.to_numeric(election['Demo date on or after Election date'])
demo_post_ed = election[['lead_city','Demo date on or after Election date']].groupby('lead_city').sum().reset_index()
demo_post_ed.rename(columns = {'lead_city':'Location','Demo date on or after Election date':'Demos done post ED'},inplace=True)

closure_post_ed = election[election['MC Election Closure Status']=='Closure post MC Election Date'] # Need to be changed
closure_post_ed = closure_post_ed[['lead_city','Closure Number']].groupby('lead_city').nunique().reset_index()
closure_post_ed.rename(columns = {'lead_city':'Location','Closure Number':'Closure post election date'},inplace=True)

closure_post_ed_fc = election[election['MC Election Closure Status']=='Closure post MC Election Date'] # Need to be changed
closure_post_ed_fc['Sold Flats'] = pd.to_numeric(closure_post_ed_fc['Sold Flats'])
closure_post_ed_fc = closure_post_ed_fc[['lead_city','Sold Flats','Closure Number','society_name']].drop_duplicates()
closure_post_ed_fc = closure_post_ed_fc[['lead_city','Sold Flats']].groupby('lead_city').sum().reset_index()
closure_post_ed_fc.rename(columns = {'lead_city':'Location','Sold Flats':'Closure post election date (FC)'},inplace=True)

closure_post_ed_fc

,Location,Closure post election date (FC)


In [ ]:
election_final = pd.merge(election_count,pd.merge(post_election_count,demo_post_ed,how='outer',on='Location'),how='outer',on='Location')
election_final = pd.merge(election_final,pd.merge(closure_post_ed,closure_post_ed_fc,how='outer',on='Location'),how='outer',on='Location').fillna(0)
election_final

,Location,Election date in this month,Activity post election date (same month),Demos done post ED,Closure post election date,Closure post election date (FC)
0,Ahmedabad,1,0.0,0.0,0.0,0.0
1,Bangalore,22,10.0,3.0,0.0,0.0
2,Hyderabad,7,2.0,0.0,0.0,0.0
3,Mumbai,2,2.0,0.0,0.0,0.0
4,Navi Mumbai,1,0.0,0.0,0.0,0.0
5,Noida,7,5.0,0.0,0.0,0.0
6,Pune,30,13.0,12.0,0.0,0.0


In [ ]:
review10 = pd.merge(review9,election_final,on='Location',how='outer').fillna(0)
review10 = review10[review10['Total flats']!=0]

In [ ]:
review10.columns

Index(['Location', 'Budgeted HC (TSM+AASM)',
       'TSMs + AASMs at start of the month (on Field)',
       'ASMs at start of the month', 'CHs/DCHs at start of the month',
       'Total flats', 'VMS Sold flats', 'ERP Sold flats',
       'VMS Onboarded Flats', 'ERP Onboarded Flats', 'VMS Onbaording %',
       'ERP Onbaording %', 'New+Winback VMS Only', 'New+Winback ERP Only',
       'New+Winback Both VMS + ERP', 'New+Winback Builders',
       'New+Winback Flats', 'Productivity New+Winback (flats)',
       'Productivity New+Winback (Including ASM)', 'Overall Productivity',
       'Winback Flats', 'Winback%', 'Upsell Flats', 'Upsell%', 'Target',
       'Total Closure', 'HO IP', 'HO Ach%', 'Month', 'Year', 'TSM Count',
       'AASM Count', 'TSM Count (Earners)', 'AASM Count (Earners)',
       'Dedicated ERP TSMs', 'Dedicated Central TSMs', 'IP/FC',
       'IP Target/TSM', 'IP Achievement/TSM', 'Number of Earners (TSM+AASM)',
       '% of Earners', 'New+Winback Societies', 'Total societies'

In [ ]:
master_review = review10[['Location','Month','Year','Budgeted HC (TSM+AASM)','TSMs + AASMs at start of the month (on Field)','ASMs at start of the month', 'CHs/DCHs at start of the month',
       'Total flats', 'VMS Sold flats', 'ERP Sold flats','VMS Onboarded Flats', 'ERP Onboarded Flats', 'VMS Onbaording %','ERP Onbaording %', 'New+Winback VMS Only', 'New+Winback ERP Only',
       'New+Winback Both VMS + ERP','New+Winback Builders', 'New+Winback Flats','Productivity New+Winback (flats)','Productivity New+Winback (Including ASM)',
       'Overall Productivity','Winback Flats','Winback%','Upsell Flats','Upsell%','Target','Total Closure','HO IP','HO Ach%','TSM Count','AASM Count','TSM Count (Earners)','AASM Count (Earners)',
       'Dedicated ERP TSMs','Dedicated Central TSMs','IP/FC','IP Target/TSM','IP Achievement/TSM','Number of Earners (TSM+AASM)','% of Earners', 'New+Winback Societies', 'Total societies',
       'FC/Society (New+Winback)', 'Productivity (Society including upsell)','VMS Only Sold flats', 'ERP Only Sold flats', 'VMS + ERP Sold flats','VMS %', 'ERP %', 'VMS + ERP %',
       'Demo Volume', 'Demo/TSM','Total Distance', 'Meetings Fixed count', 'Meetings Fixed (FC)','Invalid Leads', 'Valid Leads', 'Valid (FC)', 'Closed Lost (Count)',
       'Demo completed Leads', 'Demo completed flats','Demo Conduction % (Leads)', 'Demo Conduction % (FC)', 'Non-Compliant','Non-Compliant%', 'ASM Demo Audits','Central Closures (FC)', 'Leads created (Total)',
       'MC Election Lead', 'Election date in this month','Activity post election date (same month)', 'Demos done post ED','Closure post election date','Closure post election date (FC)']].fillna(0)



In [ ]:
review9.columns[review9.columns.duplicated(keep=False)]

Index([], dtype='object')

In [ ]:
master_review

,Location,Month,Year,Budgeted HC (TSM+AASM),TSMs + AASMs at start of the month (on Field),ASMs at start of the month,CHs/DCHs at start of the month,Total flats,VMS Sold flats,ERP Sold flats,...,Non-Compliant%,ASM Demo Audits,Central Closures (FC),Leads created (Total),MC Election Lead,Election date in this month,Activity post election date (same month),Demos done post ED,Closure post election date,Closure post election date (FC)
0,Ahmedabad,3,2024,0,6.0,0.0,2.0,3378.0,3378.0,0.0,...,0,0.0,1104.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Bangalore,3,2024,0,15.0,3.0,2.0,4702.0,3483.0,3541.0,...,50.0%,0.0,3183.0,0.0,0.0,22.0,10.0,3.0,0.0,0.0
2,Chennai,3,2024,0,4.0,0.0,1.0,1343.0,645.0,878.0,...,0,0.0,1229.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Gurgaon,3,2024,0,13.0,0.0,1.0,6364.0,1402.0,5010.0,...,0,0.0,3124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Hyderabad,3,2024,0,11.0,2.0,1.0,4434.0,3765.0,2579.0,...,25.0%,13.0,1296.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0
5,Kolkata,3,2024,0,1.0,0.0,1.0,2054.0,1572.0,482.0,...,0,0.0,1080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Mumbai,3,2024,0,19.0,3.0,1.0,1772.0,1725.0,439.0,...,33.333333333333336%,28.0,1550.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
7,Navi Mumbai,3,2024,0,9.0,1.0,1.0,2613.0,1710.0,1219.0,...,75.0%,0.0,3873.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,Noida,3,2024,0,13.0,1.0,1.0,4075.0,4075.0,2472.0,...,66.67%,0.0,8114.0,0.0,0.0,7.0,5.0,0.0,0.0,0.0
9,Pune,3,2024,0,20.0,3.0,1.0,10614.0,9289.0,1876.0,...,37.5%,0.0,4223.0,0.0,0.0,30.0,13.0,12.0,0.0,0.0


#**Write to Sheet**

In [ ]:
from gspread_dataframe import set_with_dataframe
key = "1ZwkFWS6ho0VGYYmbV5309BA4eRHDKrQ53NwoZgkZBeI"
range = "ReviewMaster"
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
review_sheet = get_data_df(key, range)

review_sheet = review_sheet[review_sheet['Month'] != str(month)]
final_review = pd.concat([review_sheet,master_review])
#final_review

In [ ]:
final_review.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
final_review.sort_values(['Year','Month','Location'], ascending = [True,True, True], inplace=True)

In [ ]:
from gspread_dataframe import set_with_dataframe
# from gspread_formatting import *
spreadSheetId = "1ZwkFWS6ho0VGYYmbV5309BA4eRHDKrQ53NwoZgkZBeI"
rangeName = "ReviewMaster"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()

set_with_dataframe(ws, final_review)